In [1]:
import pandas as pd
import quandl
import re
import datetime
from pandas.tseries.offsets import DateOffset
import numpy as np
import itertools
import wptools

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 90)
pd.set_option('display.max_rows', None)
pd.set_option('display.min_rows', 26)
pd.set_option('display.width', 200)

api_key = 'HxxAsWWP1AJofAW8u261'
quandl.ApiConfig.api_key = api_key

In [2]:
"""
US_States = pd.read_html("https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States")
US_States = US_States[1].iloc[:,0:2]
US_States.columns = ["State", "Abbreviation"]

US_States.State = US_States.State.apply(lambda x: x.strip('[D]'))
# a link to a note in wikipedia that is irrelevant to our use-case, so it is removed.

US_States.set_index('State', inplace=True)
"""
# US_States

'\nUS_States = pd.read_html("https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States")\nUS_States = US_States[1].iloc[:,0:2]\nUS_States.columns = ["State", "Abbreviation"]\n\nUS_States.State = US_States.State.apply(lambda x: x.strip(\'[D]\'))\n# a link to a note in wikipedia that is irrelevant to our use-case, so it is removed.\n\nUS_States.set_index(\'State\', inplace=True)\n'

In [3]:
"""
for index, row in US_States.iterrows():
    if index == 'Alabama':
        Temp = quandl.get("FMAC/HPI_" + row.Abbreviation)
        Temp.columns = [index, index]
        SA_Data = pd.DataFrame(Temp.iloc[:,0], index = Temp.index)
        NSA_Data = pd.DataFrame(Temp.iloc[:,1], index = Temp.index)
    else:
        Temp = quandl.get("FMAC/HPI_" + row.Abbreviation)
        Temp.columns = [index, index]
        SA_Data[index] = Temp.iloc[:,0]
        NSA_Data[index] = Temp.iloc[:,1]
"""
# Later we pickle the data so this gathering step is obsolete
# SA_Data

'\nfor index, row in US_States.iterrows():\n    if index == \'Alabama\':\n        Temp = quandl.get("FMAC/HPI_" + row.Abbreviation)\n        Temp.columns = [index, index]\n        SA_Data = pd.DataFrame(Temp.iloc[:,0], index = Temp.index)\n        NSA_Data = pd.DataFrame(Temp.iloc[:,1], index = Temp.index)\n    else:\n        Temp = quandl.get("FMAC/HPI_" + row.Abbreviation)\n        Temp.columns = [index, index]\n        SA_Data[index] = Temp.iloc[:,0]\n        NSA_Data[index] = Temp.iloc[:,1]\n'

In [4]:
# NSA_Data

In [5]:
"""
NSA_Data.to_pickle('Pickles/NSA_Data.pickle')
SA_Data.to_pickle('Pickles/SA_Data.pickle')
US_States.to_pickle('Pickles/US_States.pickle')
"""

"\nNSA_Data.to_pickle('Pickles/NSA_Data.pickle')\nSA_Data.to_pickle('Pickles/SA_Data.pickle')\nUS_States.to_pickle('Pickles/US_States.pickle')\n"

In [6]:
NSA_Data = pd.read_pickle('Pickles/NSA_Data.pickle')
SA_Data = pd.read_pickle('Pickles/SA_Data.pickle')
US_States = pd.read_pickle('Pickles/US_States.pickle')

In [7]:
Hurricanes = pd.read_html("https://en.wikipedia.org/wiki/List_of_Category_5_Atlantic_hurricanes",header=[0],flavor='bs4')
Hurricanes = Hurricanes[1]
Hurricanes.set_index("Name", inplace = True)

## Changes to be made:
1. Getting both dates into the "yyyy-mm" format, as the HPI data includes only the last day of the month.
1. Getting the Damage information from wikipedia
1. Narrowing down the Areas affected to those in the US and specifically which states involved which hurricane (Manual labour)
1. Assigining an "Overall Strength" feature which may combine different damage reports.

In [8]:
Hurricanes

,Dates as aCategory 5,Duration as aCategory 5,Sustainedwind speeds,Pressure,Areas affected,Deaths,Damage(USD),Refs
Name,,,,,,,,
"""Cuba""","October 19, 1924",12 hours,165 mph (270 km/h),910 hPa (26.87 inHg),"Central America, Mexico, CubaFlorida, The Bahamas",90,NaN,[12]
"""San Felipe IIOkeechobee""","September 13–14, 1928",12 hours,160 mph (260 km/h),929 hPa (27.43 inHg),"Lesser Antilles, The BahamasUnited States East Coast, Atlantic Canada",4000,$100 million,NaN
"""Bahamas""","September 5–6, 1932",1 day,160 mph (260 km/h),921 hPa (27.20 inHg),"The Bahamas, Northeastern United States",16,NaN,NaN
"""Camagüey""","November 5–8, 1932",3 days 6 hours,175 mph (280 km/h),915 hPa (27.02 inHg),"Lesser Antilles, Jamaica, Cayman IslandsCuba, The Bahamas, Bermuda",3103,$40 million,NaN
"""Cuba–Brownsville""","August 30, 1933",12 hours,160 mph (260 km/h),930 hPa (27.46 inHg),"The Bahamas, Cuba, FloridaTexas, Tamaulipas",179,$27.9 million,NaN
"""Tampico""","September 21, 1933",12 hours,160 mph (260 km/h),929 hPa (27.43 inHg),"Jamaica, Yucatán Peninsula",184,$5 million,NaN
"""Labor Day""","September 3, 1935",18 hours,185 mph (295 km/h),892 hPa (26.34 inHg),"The Bahamas, Florida, GeorgiaThe Carolinas, Virginia",408,NaN,NaN
"""New England""","September 19–20, 1938",18 hours,160 mph (260 km/h),940 hPa (27.76 inHg),"Eastern United States, Southwestern Quebec",682,$306 million,NaN
Carol,"September 3, 1953",12 hours,160 mph (260 km/h),929 hPa (27.43 inHg),"Bermuda, New England, Atlantic Canada",5,$2 million,NaN
